In [16]:
import selenium
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import random
from fake_useragent import UserAgent

# Read company list from .csv file

In [17]:
df = pd.read_excel('C:/Users/Acer/Downloads/google_med_equip_comp_list.xlsx')
df

,Company Name
0,MEDIEN
1,24 Lab Sdn Bhd
2,3A Pharma Sdn Bhd
3,3D Instrumentation Sdn Bhd
4,3D Ortho LifeTech Sdn Bhd
...,...
1850,Zoetis Malaysia Sdn. Bhd.
1851,Zolwira Care Medical Supplies
1852,Zontron Pharmaceuticals Sdn Bhd
1853,Zperkhidmatan Teraju Sdn Bhd


In [33]:
comp_list = list(df['Company Name'])
comp_list[1650:]

['Thoong Yin Medical Hall',
 'Thoyyib Pharma Sdn. Bhd.',
 'Three K - Tongkat Ali Extract Plus',
 'Three Sc Healthcare Supplies',
 'THROZZ MALAYSIA',
 'Thye Pharma Sdn Bhd',
 'TIGAS PHARMACY',
 'Time Healthcare Solutions Sdn Bhd',
 'Ting',
 'Tiong Yong Tung',
 'Tismed Sdn Bhd',
 'TJ-Pharma',
 'TL Healthcare Sdn. Bhd.',
 'Tohkemy (Asia) Sdn. Bhd.',
 'Tomoe Malaysia',
 'Tong Xing Yeo Hang Sdn. Bhd.',
 'Tongkah Medivest Sdn. Bhd.',
 'Tonik Medical Sdn. Bhd.',
 'Top Glove Sdn Bhd',
 'Top Glove Tower',
 'Top Medic Supplies',
 'Total Medical Supplies Sdn Bhd',
 'Touch screen world',
 'TP Care Sdn Bhd',
 'TP Pharmacy (Puchong)',
 'TPE Healthcare Sdn Bhd',
 'TPM Biotech Sdn. Bhd.',
 'TPX Medik Sdn. Bhd.',
 'Transfield Medical Products',
 'Transmedic Healthcare Sdn. Bhd.',
 'TRB Chemedica Malaysia Sdn Bhd',
 'Treasure City Motor | Jerantut',
 'Tribex Sdn Bhd',
 'Trimendous (M) Sdn. Bhd.',
 'Tringo Medic Supplies',
 'Trinity Pharmacy Lundu',
 'Trio Pharma Healthcare Sdn Bhd',
 'Trucare Pharmacy S

# Selenium

In [34]:
def apply_random_wait(count):
    """
    Apply some random wait time 
    """
    if count % 100 == 0 and count != 0:
        time.sleep(random.randint(60, 120))
    elif count % 10 == 0 and count != 0:
        time.sleep(random.randint(15, 20))
    else:
        time.sleep(random.randint(7, 15))

In [35]:
chrome_path = r<path-to-your-chromedriver>
opt = webdriver.ChromeOptions()
ua = UserAgent()
user_agent = ua.random
opt.add_argument(f'--user-agent={user_agent}')
service = Service(executable_path=chrome_path)
driver = webdriver.Chrome(service=service,options=opt)
base_url = 'https://www.mydata-ssm.com.my/login'
driver.get(base_url)
wait = WebDriverWait(driver, 30)
driver.maximize_window()
username = driver.find_element(By.XPATH,'//*[@id="mat-input-0"]')
username.send_keys(<your-username>)
password = driver.find_element(By.XPATH,'//*[@id="mat-input-1"]')
password.send_keys(<your-password>)
login = driver.find_element(By.XPATH,'//*[@id="recaptcha2"]')
login.click()
try:
   time.sleep(5)
   webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
except:
   pass
time.sleep(5)

## Access company name

In [36]:
data=str()
comp_search = []
comp_num = []
comp_name = []
comp_type = []

for i, comp in enumerate(comp_list[1650:]):
    search = driver.find_element(By.CLASS_NAME,'form-control')
    search.send_keys(comp)
    webdriver.ActionChains(driver).send_keys(Keys.ENTER).perform()
    apply_random_wait(i)

    data += driver.page_source
    
    # Company to validate
    search_comp = comp
    comp_search.append(search_comp)

    # Company number 
    try:
        num_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-entityNumber.mat-column-entityNumber.ng-star-inserted').text.replace('Name\n','')
    except:
        num_comp = "No match"
    comp_num.append(num_comp)

    # Company name 
    try:
        name_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-name.mat-column-name.ng-star-inserted').text.replace('Name\n','')
    except:
        name_comp = "No match"
    comp_name.append(name_comp)

    # Company type
    try:
        type_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-entityType.mat-column-entityType.ng-star-inserted').text.replace('Type\n','') 
    except:
        type_comp = "No match"
    comp_type.append(type_comp)
    apply_random_wait(i)

    # Clear the search bar
    search.clear()


# Convert into dataframe 

In [37]:
len(comp_name)

205

In [38]:
len(comp_num)

205

In [ ]:
# scrape_data = pd.DataFrame()

# scrape_data['Company Verification'] = comp_search
# scrape_data['Company Name']= comp_name
# scrape_data['Company Type'] = comp_type
# scrape_data

## Example for one company (Selenium)

In [ ]:
reg_num = driver.find_element(By.CSS_SELECTOR, 'mat-row mat-cell:first-child a.ng-star-inserted').text 
reg_num

In [ ]:
name_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-name.mat-column-name.ng-star-inserted').text.replace('Name\n','')
name_comp

In [ ]:
type_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-entityType.mat-column-entityType.ng-star-inserted').text.replace('Type\n','')
type_comp

# BeautifulSoup

In [39]:
soup = BeautifulSoup(data, 'html.parser')
row_data = soup.find_all('mat-row','example-element-row')

records = []
for row in row_data:
    #comp_search = company_name
    # try:
    comp_number = row.find('a','ng-star-inserted').text.strip()
    # except: 
    #     comp_num = "No match"
    
    # try:
    comp_name = row.find('mat-cell','mat-cell cdk-column-name mat-column-name ng-star-inserted').text.strip().replace('Name','')
    # except:
    #     comp_name = "No match"

    # try:
    comp_type = row.find('mat-cell','mat-cell cdk-column-entityType mat-column-entityType ng-star-inserted').text.strip().replace('Type','')
    # except:
    #     comp_type = "No match"

    company_info = (comp_number, comp_name, comp_type) #, comp_name, comp_type

    records.append(company_info)

In [40]:
records

[('200101013408 (549165-H)', 'THOYYIB PHARMA SDN. BHD.', 'Company'),
 ('201403077093 (UT0004296-X)', 'THREE SC HEALTHCARE SUPPLIES', 'Business'),
 ('200701042423 (800455-A)', 'THYE PHARMA SDN. BHD.', 'Company'),
 ('201301035644 (1065472-P)',
  'TIME HEALTHCARE SOLUTIONS SDN. BHD.',
  'Company'),
 ('196201000092 (4608-V)',
  'TINGGI MINING & INVESTMENT COMPANY S/B',
  'Company'),
 ('197101000602 (11110-D)', 'TING MING HOI ENTERPRISE SDN. BHD.', 'Company'),
 ('197301000193 (13716-A)', 'TING HING CO SDN. BHD.', 'Company'),
 ('197301001732 (17590-P)', 'TING HING CHONG REALTY SDN. BHD.', 'Company'),
 ('197401002681 (19732-H)', 'TING FOONG TRADING SDN. BHD.', 'Company'),
 ('197501000798 (22582-H)', 'TING ING HING & SONS SDN. BHD.', 'Company'),
 ('197501003591 (25330-U)', 'TINGGI FARM SDN. BHD.', 'Company'),
 ('197601002629 (28786-T)',
  'TING PECK WUI ENTERPRISE & DEVELOPMENT SDN. BHD.',
  'Company'),
 ('197601004323 (30320-U)',
  'TINGGI CONSTRUCTION & ENTERPRISE SDN.BHD.',
  'Company'),
 (

In [41]:
len(records)

194

In [ ]:
# comp_number = []
# j = 0
# for i in range(len(comp_name)):
#     if comp_name[i] == 'No match':
#         comp_number.append('No match')
#     else:
#         comp_number.append(records[j])
#         j += 1
# comp_number

In [42]:
col = ['registration_no','company_match_ssm','company_type']
scrape_data = pd.DataFrame(records,columns=col)
scrape_data

,registration_no,company_match_ssm,company_type
0,200101013408 (549165-H),THOYYIB PHARMA SDN. BHD.,Company
1,201403077093 (UT0004296-X),THREE SC HEALTHCARE SUPPLIES,Business
2,200701042423 (800455-A),THYE PHARMA SDN. BHD.,Company
3,201301035644 (1065472-P),TIME HEALTHCARE SOLUTIONS SDN. BHD.,Company
4,196201000092 (4608-V),TINGGI MINING & INVESTMENT COMPANY S/B,Company
...,...,...,...
189,200801028585 (829912-M),ZENOTEC PHARMA SDN. BHD.,Company
190,196901000621 (8898-V),ZOETIS MALAYSIA SDN. BHD.,Company
191,199701030196 (445695-T),ZONTRON PHARMACEUTICALS SDN. BHD.,Company
192,201001006128 (890747-W),ZS MEDICAL SUPPLIERS & SERVICES SDN. BHD.,Company


In [ ]:
# scrape_data = pd.DataFrame()
# scrape_data['Company Verification'] = comp_search
# scrape_data['Registration Number'] = comp_number
# scrape_data['Company Name']= comp_name
# scrape_data['Company Type'] = comp_type
# scrape_data

In [32]:
# Read mydata_ssm.xlsx file
# mydata_df = pd.read_excel('C:/Users/Acer/Downloads/mims_ssm.xlsx')

# Concatenate 'scrape_data' dataframe with 'df'
# mydata_df = pd.concat([mydata_df, scrape_data], ignore_index=True)

# Write the updated dataframe back to 'reg_med_device_ssm.xlsx' mydata_df
scrape_data.to_excel('C:/Users/Acer/Downloads/google_med_equip_ssm2.xlsx', index=False)

## Example for one row (BeautifulSoup)

In [ ]:
row = row_data[0]
row

In [ ]:
comp_num = row.find('a','ng-star-inserted').text.strip()
comp_num

In [ ]:
comp_name = row.find('mat-cell','mat-cell cdk-column-name mat-column-name ng-star-inserted').text.strip().replace('Name', '')
#comp_name = comp_name.replace('Name', '')
comp_name

In [ ]:
comp_type = row.find('mat-cell','mat-cell cdk-column-entityType mat-column-entityType ng-star-inserted').text.strip().replace('Type', '')
#comp_type = comp_type.replace('Type', '')
comp_type